In [9]:
import pywinauto
import pyautogui
from pywinauto import Application, Desktop
import keyboard
from pywinauto.findwindows import ElementNotFoundError, ElementAmbiguousError
from pywinauto.uia_defines import NoPatternInterfaceError
import time
import os
from subprocess import Popen
import configparser
#import xlwings as xw
#import shutil
#import stopit                                                                                     
from stopit import ThreadingTimeout as Timeout
import openpyxl 
import pandas as pd
import os
from PyPDF2 import PdfFileMerger
from datetime import datetime
from tqdm import tqdm
import shutil

In [13]:
#diretorio = r'''F:\Faturas\Robo2'''
planilha_rodando = r'D:/Users/CSR342486/Desktop/Faturamento/2022/09.Setembro/FAT_FRA - P1/FAT_FRA - P1.xlsx'

def press_release(key):
    keyboard.press(key)
    keyboard.release(key)

# Configurações do aplicativo (default)

    # 'CSR957147' #'Maio#2022'
    # 'CSR957145' #'Livi@2021'
    # 'CSR351676' #'HW8U@1sc'
    # 'CSR957146' #'#Junho22'

user_crm =  'CSR957146'
user_ccs =  'CSR957146'
pw_crm =  '#Junho22'
pw_ccs =  '#Junho22'
pasta =  r'D:/Users/CSR342486/Desktop/Faturamento/2022/09.Setembro'
caminho_salvar = r'D:/Users/CSR342486/Desktop/Faturamento/2022/09.Setembro/FAT_FRA - P1'
data_ini = '01.09.2022' 
data_fim = '06.09.2022' 
count = 2
nome_relatorio = 'Relatório Impressoes Julho P2.xlsx'
# Fim das configurações

arquivo = pd.read_excel(planilha_rodando,  engine='openpyxl', converters={'ANLAGE': str, 'QMNUM': str, 'ZCGACCOUN': str})

#inicialização+
sap = Desktop(backend='uia').window(class_name="SAP_FRONTEND_SESSION")
ie = Desktop(backend="uia").window(class_name="IEFrame", visible_only=False, found_index=0)
ie_win_seg = Desktop(backend='uia').window(class_name="IEFrame", visible_only=False, found_index=0).window(class_name="#32770") #Desktop(backend='uia').window(class_name="Alternate Modal Top Most")
pdfcreator = Desktop(backend='uia').window(title="PDFCreator", visible_only=False)
pdfcreatorprint = Desktop(backend='uia').window(title="Gerenciar Trabalhos de Impressão", visible_only=False)
pdf = Desktop(backend='uia').window(class_name="AcrobatSDIWindow", visible_only=False)
destino = Desktop(backend='uia').window(title="Selecionar destino")
docs = Desktop(backend="uia").window(class_name="CabinetWClass", visible_only=False, found_index=0)
salvar = Desktop(backend='uia').window(class_name="")
interaction = ie.window(class_name="Frame Tab").window(title="Interaction Center WebClient - Windows Internet Explorer")
sessao_encerrada = ie.window(class_name="#32770", tittle="Interaction Center WebClient - Windows Internet Explorer")

In [15]:
# Garantir encerramento de processos SAP e Internet Explorer
os.system("taskkill /f /im saplogon.exe")
os.system("taskkill /f /im iexplore.exe")

# wb = openpyxl.load_workbook(filename=planilha_rodando)
# ws = wb.active
# time.sleep(5)

count_login=0
if(not(Desktop(backend='uia').window(title_re="^Imprimir.*Via$",found_index=0).exists())):

    sap = Application(backend='uia').start(r"C:/Program Files (x86)/SAP/FrontEnd/SAPgui/saplogon.exe")
    try:
        sap = Desktop(backend='uia').SAPLogon740.child_window(title="05.02 - Cosern CRM Produção", found_index=0).select()
        print('Abriu o SAP')
    except:
        print("Erro ao abrir o SAP")
    sap = Desktop(backend='uia').SAPLogon740.child_window(title="05.02 - Cosern CRM Produção", found_index=0).type_keys("{ENTER}")
    
    # os.system(r"start sapgui brnep468.neoenergia.net 01")
    time.sleep(1)
    
    sap = Desktop(backend='uia').window(class_name="SAP_FRONTEND_SESSION")
    with Timeout(60.0):
        while(True):
            try:
               sap.type_keys(user_ccs+"{TAB}"+pw_ccs+"{ENTER}")
            except:
               print("Não encontrado o login no SAP")
            else:
               print("Login no SAP realizado")
               break
    while(True):
        if(sap.child_window(auto_id="146", found_index=0).exists()):
            break
    if(not(Desktop(backend="uia").window(class_name="IEFrame", visible_only=False, found_index=0).exists())):
        app = Application(backend='uia').start(r"C:/Program Files/Internet Explorer/iexplore.exe")
    ie = Desktop(backend="uia").window(class_name="IEFrame", visible_only=False, found_index=0) 
    
    ierest_ct = 0
    while(ierest_ct<1):
        try:
            time.sleep(0.1)
            ie_win_seg.child_window(title="Ir para a sua home page", found_index=0).click_input()
        except:
            print("Não apareceu mensagem de restaurar sessão")
            ierest_ct += 1
        else:
            time.sleep(1)  
    ie.maximize()
    time.sleep(5)

    # OLHAR ESSA PARTE DEPOIS
    # if(Desktop(backend="uia").window(class_name="IEFrame", visible_only=False, found_index=0).window(title=" Windows Internet Explorer").exists()):
        # sessao_encerrada.child_window(title="Fechar", found_index=0).click()
        # time.sleep(5)
    #if(ie.child_window(class_name="TabWindowClass", title="Iniciar Sessão - Windows Internet Explorer", found_index=0).exists()): #Portal dos Colaboradores do Iberdrola - Internet Explorer
    
    ie.child_window(class_name="Edit", title="Endereço", control_type="Edit").set_text(r"http://brnep485.neoenergia.net:8002/sap/bc/bsp/sap/crm_ic/default.htm")
    press_release('enter')

    count_login=0
    while(True):
        count_login += 1
        try:
            ie_win_seg.child_window(title="Nome de usuário").set_text(user_crm)
            ie_win_seg.child_window(title="Senha").set_text(pw_crm)
            press_release('enter')
        except:
            print("Não achou o autenticador de usuário - Internet Explorer")
            if(count_login == 5):
                break 
        else:
            print("Achou o campo de autenticação de usuário - Internet Explorer")
            break
        time.sleep(1)

    lp_ct = 0
    while(True):
        try:
            time.sleep(1)
            ie.child_window(title="ZPERDAS", found_index=0).click_input()
            print("ZPerdas selecionado")
            break
        except:
            print("ZPerdas nao encontrado")
            if(lp_ct > 5):
                break
            lp_ct += 1 
    
    lista_cc = []
    lista_nota = []
    lista_status = []
    lista_tipo = []
    #while(True):

    #lista_conta = ['007009166801', '007009166801', '007012692590', '000855294583']
    #lista_qmnum = ['004401163436', '004401163436', '004401153808', '004401162526']
    
    for lin, cc in tqdm(enumerate(arquivo['ZCGACCOUN'])):  #lista_conta #arquivo.iterrows()
        # cc = dados['ZCGACCOUN']
        nota = arquivo['QMNUM'][lin] #lista_qmnum[lin] # dados['QMNUM']

        lista_nota.append(nota)
        lista_cc.append(cc)

        print(lin, cc, nota)

        novo_arq = caminho_salvar+'Docs_unidos/'+nota+'.pdf'
        ant_caminho_arqs = caminho_salvar+'Docs_separados/'

        if os.path.isfile(novo_arq):
            print('A comunicação da nota {} já existe'.format(nota))
            lista_status.append('OK')
            df_final = pd.DataFrame(list(zip(lista_cc,lista_nota,lista_status,lista_tipo)), columns=['Conta Contrato','Nota','Status Download','Status Nota']) 
            df_final.to_excel(caminho_salvar+'Docs_unidos/'+nome_relatorio)
            pass
        else:
            # podeunir=0
            # while(podeunir == 0):
                
            ie.child_window(title="Interaction Center WebClient",found_index=0).click_input()
            interaction.child_window(title="Encerrar", found_index=0).click_input()
            
            while(True):
                try:
                    ie.child_window(title="Data de nascimento", control_type="Edit", found_index=1).type_keys("{TAB}{TAB}{TAB}{TAB}")
                except ElementAmbiguousError:
                    print('Ainda não achou o campo Data de Nascimento')
                else:
                    print('Achou o campo Data de Nascimento')
                    break
            ie.child_window(title="Data de nascimento", control_type="Edit", found_index=1).type_keys("{TAB}{TAB}{TAB}{TAB}")
            time.sleep(2)
            keyboard.write(cc)

            try:
                ie.child_window(title="Procurar conta", found_index=1).select()
            except:
                print("Sem sucesso!")

            try:
                ie.child_window(title="Procurar conta", found_index=1).type_keys("{ENTER}")
                time.sleep(3)
            except:
                print("Sem sucesso!")
            while(True):
                try:
                    ie.child_window(title="Procurar conta", found_index=1).select()
                except ElementNotFoundError:
                    print('Acabou de procurar o cliente')
                    break
                except NoPatternInterfaceError:
                    print('Acabou de procurar o cliente')
                    break
                else:
                    print('Ainda está procurando')
            
            try:
                ie.child_window(title="Consulta de Atividades", found_index=0).click_input()
            except:
                print("Tentando consultar Atividades")

            while(True):
                try:
                    ie.child_window(title="Proc.", found_index=0).type_keys("+{TAB}+{TAB}+{TAB}")
                except ElementNotFoundError:
                    print('Procurando processo')
                    press_release('enter')
                    ie.child_window(title="Consulta de Atividades", found_index=0).click_input()
                else:
                    print('Acabou os processos')
                    break
            time.sleep(4)
            keyboard.write(nota)
            press_release('enter')
            time.sleep(4)
            
            count_comunicao = 0
            while(True):
                try:
                    ie.child_window(title="P-Comuni. de Débitos", found_index=0).type_keys("{ENTER}")
                    if(count_comunicao>3):
                        ie.child_window(title="Proc.", found_index=0).type_keys("+{TAB}+{TAB}+{TAB}")
                    count_comunicao += 1
                except:
                    print("Problema ao inserir a nota")
                else:
                    break
                if count_comunicao > 5:
                    break

            if(ie.child_window(title="Status", control_type="ComboBox").selected_text() != "Fatura Simulada"):
                
                # ie.child_window(title="Status", control_type="ComboBox").click_input()
                # ie.child_window(title="Status", control_type="ComboBox").type_keys("Comunicação")
                # ie.child_window(title="Status", control_type="ComboBox").type_keys("{ENTER}")

                if(ie.child_window(title="Status", control_type="ComboBox").selected_text() == "Concluir Proc. Imp. Cobrança"):
                    time.sleep(2)
                    ie.child_window(title="Status", control_type="ComboBox").click_input()
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{DOWN}")
                    time.sleep(2)
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{ENTER}")
                    time.sleep(3)
                    lista_tipo.append('Fatura Simulada')

                if(ie.child_window(title="Status", control_type="ComboBox").selected_text() == "Fatura Gerada"):

                    ie.child_window(title="Status", control_type="ComboBox").click_input()
                    time.sleep(2)
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{DOWN}")
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{ENTER}")
                    time.sleep(2)
                    ie.child_window(title="Status", control_type="ComboBox").click_input()
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{DOWN}")
                    ie.child_window(title="Status", control_type="ComboBox").type_keys("{ENTER}")
                    time.sleep(3)
                    lista_tipo.append('Fatura gerada')
            else:
                lista_tipo.append('Fatura Simulada')

            ie.child_window(title="Atividades subseqüentes e ações",found_index = 0).type_keys("{TAB}{TAB}{TAB}{TAB}{TAB}")
            if(ie.child_window(title="Reimprimir Comunicação de Débitos", found_index=0).exists()):
                ie.child_window(title="Reimprimir Comunicação de Débitos", found_index=0).type_keys("{TAB}{ENTER}")
            else:
                ie.child_window(title="Atividades subseqüentes e ações",found_index = 0).type_keys("{TAB}{TAB}{TAB}{TAB}{TAB}{TAB}{ENTER}")
            
            while (pyautogui.pixel(223,462) != (242, 242, 242)):
                time.sleep(1)

            count_avancar = 0
            while(True):
                try:
                    pyautogui.click(201,550)
                    press_release('tab')
                    press_release('enter')
                    #ie.window(class_name="Frame Tab").window(title="Interaction Center WebClient - Windows Internet Explorer").child_window(title="Avançar ", found_index=0).type_keys("{ENTER}")
                    print('Avançou')
                    break
                except:
                    print("Tentando avançar")
                count_avancar += 1
                if (count_avancar > 5 ):
                    break
            # time.sleep(7)

            while (pyautogui.pixel(224,439) != (163, 193, 228)):
                time.sleep(1)
                
            count_check = 0
            while(True):
                try:
                    try:
                        ie.window(class_name="Frame Tab").window(title="Interaction Center WebClient - Windows Internet Explorer").child_window(title="Avançar", found_index=1).type_keys("{ENTER}")
                        break
                    except:
                        for n in range(0,1):
                            press_release('tab')
                        time.sleep(1)
                        press_release('enter')
                        print('Deu check')
                        break
                except:
                    print('Tentando dar um check')
                count_check += 1
                if (count_check > 5 ):
                    break

            for k in range(0,3):
                time.sleep(5)
                count_pre_visualizacao = 0
                while(True):
                    try: 
                        try:
                            for n in range(0,20):
                                press_release('tab')
                            press_release('enter')
                            print('Clicou em pré-visualização')
                            break
                        except:
                            # pyautogui.hotkey('ctrl','a')
                            # pyautogui.write('Local Printer')
                            interaction.child_window(title="   Pré-visualização ").double_click_input()
                            print('Clicou em pré-visualização')
                            break
                    except:
                        print("Procurando Pre Visualizacao")
                        ie.maximize()
                    count_pre_visualizacao += 1
                    if (count_pre_visualizacao > 5 ):
                        break

                while pyautogui.pixel(367,463) != (183, 183, 183):
                    time.sleep(1)
                
                pyautogui.click(201,550)
                pyautogui.hotkey('ctrl','shift','s')
                time.sleep(4)
                # while(not(ie.window(class_name="#32770", title='Salvar como').exists())):
                #     try:
                #         time.sleep(1)
                #     except:
                #         time.sleep(1)
                #         ie.window(class_name="#32770", title='Salvar como').child_window(title=r"Endereço: D:\Users\ETR351113\Downloads", control_type="ToolBar").type_keys(caminho_salvar)
                #         break
                
                #### SALVAR (MANUAL) ####
                # pyautogui.doubleClick(374,396)
                # press_release('backspace')
                # for n in range(0,11):
                #     press_release('tab')
                # press_release('backspace')
                # pyautogui.write(nota+'('+str(k)+')')
                # press_release('enter')
                # time.sleep(2)

                if(ie.window(class_name="#32770", title='Salvar como').exists()):
                    press_release('tab')
                    press_release('enter')        
                else:
                    pass  
                time.sleep(3)
                try:
                    if os.path.isfile(caminho_salvar+'smart.pdf'):
                        os.rename(caminho_salvar+'smart.pdf',caminho_salvar+nota+'('+str(k)+').pdf')
                except:
                    pass
                os.system("taskkill /f /im AcroRd32.exe")
                time.sleep(5)
                ie.child_window(title=" Voltar ").click_input()
                print('Clicou em voltar')
                while True:
                    if(ie.child_window(title=" Voltar ").exists()):
                        time.sleep(1)
                    else:
                        break
            
            pdfs = [nota+'(0).pdf', nota+'(1).pdf', nota+'(2).pdf']
            try:
                print('Etapa de juntar os arquivos...')
                if os.path.isdir(caminho_salvar+'Docs_unidos'):
                    pass
                else:
                    os.mkdir(caminho_salvar+'Docs_unidos')
                
                merger = PdfFileMerger()
                merger.append(caminho_salvar+pdfs[0], pages=(2,3))
                merger.append(caminho_salvar+pdfs[1])
                merger.append(caminho_salvar+pdfs[2])
                merger.write(novo_arq)
                merger.close
                print('A comunicação da nota {} foi salva e mesclada'.format(nota))
                lista_status.append('OK')
                try:
                    for indice in range(0,3):
                        os.remove(caminho_salvar+pdfs[indice])
                except Exception as e:
                    print(e)
                    pass
                
                if os.path.isdir(caminho_salvar+'Docs_separados'):
                    pass
                else:
                    os.mkdir(caminho_salvar+'Docs_separados')

                for pdf in pdfs:    
                    try:
                        shutil.move(caminho_salvar+pdf,ant_caminho_arqs+pdf)
                    except:
                        pass
            except:
                print('Deu erro')
                lista_status.append('ERRO')

            print(lista_tipo)
            df_final = pd.DataFrame(list(zip(lista_cc,lista_nota,lista_status,lista_tipo)), columns=['Conta Contrato','Nota','Status Download','Status Nota']) 
            df_final.to_excel(caminho_salvar+'Docs_unidos/'+nome_relatorio)
            
        
    

Abriu o SAP
Login no SAP realizado
Não apareceu mensagem de restaurar sessão
Achou o campo de autenticação de usuário - Internet Explorer
ZPerdas selecionado


0it [00:00, ?it/s]

0 007012826510 004401133519
A comunicação da nota 004401133519 já existe


2it [00:02,  1.24s/it]

1 007012626899 004401133439
A comunicação da nota 004401133439 já existe
2 007011960736 004401135526
A comunicação da nota 004401135526 já existe


...

191 007011709235 004401180717
A comunicação da nota 004401180717 já existe
192 000512951015 004401179067
A comunicação da nota 004401179067 já existe
193 007012379956 004401147936


193it [00:36, 15.62it/s]

Achou o campo Data de Nascimento
Sem sucesso!
Acabou de procurar o cliente
Acabou os processos
Avançou
Clicou em pré-visualização
Clicou em voltar


193it [04:12,  1.31s/it]


KeyboardInterrupt: 

## Ajustes 

In [1]:
notas = input("Insira uma lista de notas separadas por espaço para faturar: ")
list_notas  = notas.split()
print(list_notas)

Insira uma lista de notas separadas por espaço para faturar: 004401077585 004401077585 004401077585
['004401077585', '004401077585', '004401077585']
